In [1]:
import tabulate

def print_data_table(dataset, exclude_columns=[]):
    dataset = [
        {key: value for key, value in entry.items() if key not in exclude_columns}
        for entry in dataset
    ]
    header = dataset[0].keys()
    rows = [x.values() for x in dataset]
    print(tabulate.tabulate(rows, header))

In [ ]:
import json

def parse_power_json(file_path):
    data = {}
    with open(file_path) as f:
        data = json.load(f)

    averages = []
    for document in data:
        average = document.get("energy", {}).get("average")
        if average is not None:
            averages.append(average)
    
    return averages

In [ ]:
REPETITIONS = 30

power_data = []

exp_names = [
    'both_funcs_on_rpi',
    'both_funcs_on_vm',
    'first_func_on_rpi_second_func_on_vm',
    'first_func_on_vm_second_func_on_rpi'
]

for exp in exp_names:
    exp_ttc_data = {
        "exp_name": exp,
        "power_summary_file": exp + '/power_monitor/summary.json',           # Location of summary.json file
        "avg_power_measurements": []             # For box plot
    }
    power_data.append(exp_ttc_data)


for exp in power_data:
    exp['avg_power_measurements'] = parse_power_json(exp['power_summary_file'])


print_data_table(power_data, exclude_columns=['power_summary_file'])

all_data_lists = []
for exp in power_data:
    all_data_lists.append(exp['avg_power_measurements'])

In [ ]:
import json
from statistics import mean

data = {
    'x_cores': [
        {
            'exp_name': power_data[0].get('exp_name'),
            'avg_power': mean(power_data[0].get('avg_power_measurements'))
        },
        {
            'exp_name': power_data[1].get('exp_name'),
            'avg_power': mean(power_data[1].get('avg_power_measurements'))
        },
        {
            'exp_name': power_data[2].get('exp_name'),
            'avg_power': mean(power_data[2].get('avg_power_measurements'))
        },
        {
            'exp_name': power_data[3].get('exp_name'),
            'avg_power': mean(power_data[3].get('avg_power_measurements'))
        },
    ]
}

json_str = json.dumps(data)
json_str

In [ ]:
import matplotlib.pyplot as plt

def show_combined_box_plot(x_labels, y_label, title, data):
    plt.boxplot(data, patch_artist=True, labels=x_labels)
    plt.yscale("log")
    plt.title(title)
    plt.ylabel(y_label)
    plt.show()

show_combined_box_plot(
    [
        'RPI\nonly', 
        'Two functions\non VMpub',
        'Feature\nextraction\non RPI,\nclassification\non VMpub',
        'Feature\nextraction\non VMpub,\nclassification\non RPI'
    ], 
    'Energy (Wh)',
    'Energy data (x active cores)', 
    all_data_lists
)